#                           Предсказание кассовых сборов фильмов

Для предсказания будем использовать датасет с Каггла: TMDB Box Office Prediction - https://www.kaggle.com/c/tmdb-box-office-prediction

In [1]:
%config IPCompleter.greedy=True

In [2]:
# !pip install -r requirements.txt

Заимпортим все необходимые библиотеки

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("data/train.csv")

In [6]:
df

## Часть 1: предобработка данных и подготовка к обучению

Удалим ненужные колонки: пока что считаем, что название фильма нерелевантно, лишние id нам не нужны, как и ссылки

In [7]:
del df['id']
del df['homepage']
del df['imdb_id']
del df['original_title']
del df['title']
del df['poster_path']
del df['status']

Для нас не будет важно, в какой коллекции находится фильм, будет достаточно того, что он находится в *какой-то*

In [9]:
df["belongs_to_collection"].replace([np.nan, -np.inf], 0, inplace=True)
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: int(isinstance(x, str)))

Заметим, что у нас много признаков находятся в строковом представлении словаря. Превратим их в нормальные словари

In [10]:
import ast
bad_count = 0

def convert_bad_features(list_of_strings) -> list[dict]:
    """Converts list of dicts in its string representation to a normal Python list of normal Python dicts."""
    if not isinstance(list_of_strings, str):
        return []
    else:
        return ast.literal_eval(list_of_strings)
    
df['genres'] = df['genres'].apply(convert_bad_features)
df['production_companies'] = df['production_companies'].apply(convert_bad_features)
df['production_countries'] = df['production_countries'].apply(convert_bad_features)
df['spoken_languages'] = df['spoken_languages'].apply(convert_bad_features)
df['Keywords'] = df['Keywords'].apply(convert_bad_features)
df['cast'] = df['cast'].apply(convert_bad_features)
df['crew'] = df['crew'].apply(convert_bad_features)

Однако из этих словарей нам нужны не все значения. 

Из жанров нам достаточно их названия, которое мы закодируем числом, а позже применим One Hot Encoding

In [11]:
df['genres'] = df['genres'].apply(lambda x: [d['name'] for d in x])

In [12]:
all_genres = sorted(list(set([g for genre_list in df['genres'] for g in genre_list])))
GENRE_TO_INT = {g : i for i, g in enumerate(all_genres)}
INT_TO_GENRE = {i : g for i, g in enumerate(all_genres)}

In [13]:
df['genres'] = df['genres'].apply(lambda x: [GENRE_TO_INT[g] for g in x])

Таким же образум поступим с языками

In [14]:
all_languages = [lang.lower() for lang in df['original_language']]
# ещё учтём, что в колонке 'spoken_languages' есть языки, которых нет в 'original_language'
all_languages += [l['iso_639_1'].lower() for sp_lang_list in df['spoken_languages'] for l in sp_lang_list]
all_languages = sorted(list(set(all_languages)))
LANG_TO_INT = {g : i for i, g in enumerate(all_languages)}
INT_TO_LANG = {i : g for i, g in enumerate(all_languages)}

In [15]:
df['original_language'] = df['original_language'].map(LANG_TO_INT)

In [16]:
df['spoken_languages'] = df['spoken_languages'].apply(lambda x: [LANG_TO_INT[d['iso_639_1']] for d in x])

Преобразуем страны производства

In [17]:
all_countries = [ctry['iso_3166_1'].upper() for ctry_list in df['production_countries'] for ctry in ctry_list]
all_countries = sorted(list(set(all_countries)))
COUNTRY_TO_INT = {g : i for i, g in enumerate(all_countries)}
INT_TO_COUNTRY = {i : g for i, g in enumerate(all_countries)}

In [18]:
df['production_countries'] = df['production_countries'].map(lambda x: [COUNTRY_TO_INT[d['iso_3166_1']] for d in x])

Даже в этом сравнительно небольшом датасете очень много разных людей в cast и crew, как и кинокомпаний. Вряд ли получится адекватно учесть все эти признаки без переобучения, поэтому уберём их совсем.

In [19]:
del df['production_companies']
del df['cast']
del df['crew']

Из даты выпуска оставим только год, так как остальное будет пренебрежимо слабо влиять на предсказание

In [20]:
def release_date_to_year(date: str):
    year = int(date[-2:])
    if year > 20: # Dataset is from 5 years ago
        return (1900 + year)
    else:
        return (2000 + year)

In [21]:
df['release_year'] = df['release_date'].apply(release_date_to_year)
del df['release_date']

Предположительно, все текстовые фичи (overview, tagline, keywords) влияют на кассовые сборы примерно одинаково. Сконкатенируем их, и сделаем фичу description.

*\* tagline — рекламный слоган, сопровождающий картину*

In [23]:
df['Keywords'] = df['Keywords'].apply(lambda x: ' '.join([w['name'] for w in x]))

In [24]:
df['description'] = df.agg(lambda x: f"{x['overview']}" if isinstance(x['overview'], str) else "", axis=1)
df['description'] = df.agg(lambda x: f"{x['description']} {x['tagline']}" if isinstance(x['tagline'], str) else x['description'], axis=1)
df['description'] = df.agg(lambda x: f"{x['description']} {x['Keywords']}" if isinstance(x['Keywords'], str) else x['description'], axis=1)
del df['overview']
del df['tagline']
del df['Keywords']

In [25]:
df

,belongs_to_collection,budget,genres,original_language,popularity,production_countries,runtime,spoken_languages,revenue,release_year,description
0,1,14000000,[3],12,6.575393,[71],93.0,[12],12314651,2015,"When Lou, who has become the ""father of the In..."
1,1,40000000,"[3, 6, 7, 14]",12,8.248895,[71],113.0,[12],95149435,2004,Mia Thermopolis is now a college graduate and ...
2,0,3300000,[6],12,64.299990,[71],105.0,[12],13092000,2014,"Under the direction of a ruthless instructor, ..."
3,0,1200000,"[17, 6]",24,3.174936,[34],122.0,"[12, 24]",16000000,2012,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...
4,0,0,"[0, 17]",34,1.148070,[41],118.0,[34],3923970,2009,Marine Boy is the story of a former national s...
...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,"[3, 14]",12,9.853270,[71],102.0,[12],1596687,1994,Military men Rock Reilly and Eddie Devane are ...
2996,0,0,"[6, 12]",64,3.727996,"[19, 64]",102.0,[64],180590,2013,Three girls in 1980s Stockholm decide to form ...
2997,0,65000000,"[4, 0, 13, 17]",12,14.482345,[71],120.0,[12],89456761,1996,"Samantha Caine, suburban homemaker, is the ide..."
2998,0,42000000,"[3, 14]",12,15.725542,[71],90.0,[12],171963386,2004,Reuben Feffer is a guy who's spent his entire ...


Готово (на самом деле ещё нужен One Hot Encoding).

Сохраним это, чтобы не потерять

In [29]:
df.to_csv('data/preprocessed_train.csv')

Колонка revenue является нашим таргетом

In [26]:
data = df.drop("revenue", axis= 1)
target = df["revenue"]

In [27]:
# all_genres = set([g for genre_list in df['genres'] for g in genre_list])
# all_ = set([g for genre_list in df['genres'] for g in genre_list])

In [28]:
# len(all_genres)